In [84]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import os
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score
import joblib

from sklearn.linear_model import LogisticRegression

In [75]:
def calculate_angle(center_landmark, start_landmark, end_landmark):

    center = np.array(center_landmark)
    start = np.array(start_landmark)
    end = np.array(end_landmark)
    

    vector1 = start - center
    vector2 = end - center
    

    angle_radians = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
    

    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees


def draw_selected_landmarks_on_image(rgb_image, landmarks, selected_landmarks, connections):

    annotated_image = np.copy(rgb_image)


    for index in selected_landmarks:
        landmark = landmarks.landmark[index]
        x, y, z = landmark.x, landmark.y, landmark.z

        x_pixel, y_pixel = int(x * annotated_image.shape[1]), int(y * annotated_image.shape[0])
        cv2.circle(annotated_image, (x_pixel, y_pixel), radius=5, color=(0, 255, 0), thickness=-1)
    for connection in connections:
        start_index, end_index = connection
        start_landmark = landmarks.landmark[start_index]
        end_landmark = landmarks.landmark[end_index]
        start_x, start_y = int(start_landmark.x * annotated_image.shape[1]), int(start_landmark.y * annotated_image.shape[0])
        end_x, end_y = int(end_landmark.x * annotated_image.shape[1]), int(end_landmark.y * annotated_image.shape[0])
        cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
            

    return annotated_image

In [76]:
mp_pose = mp.solutions.pose
selected_landmarks_indices = [16,14,12,24,26,28,32]
selected_landmarks_connections = [(16,14),(14,12),(24,26),(26,28),(12,24),(24,28),(28,32)]

In [77]:
data=pd.read_csv('hip-train.csv')

X = data[['angle']]
y = data['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

clf = RandomForestClassifier(random_state=42)

In [78]:
clf.fit(X_train, y_train)

train_preds = clf.predict(X_train)
val_preds = clf.predict(X_val)
test_preds = clf.predict(X_test)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
test_acc = accuracy_score(y_test, test_preds)

conf_matrix = confusion_matrix(y_test, test_preds)

precision = precision_score(y_test, test_preds)
recall = recall_score(y_test, test_preds)

print(f"Training accuracy: {train_acc:.4f}")
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print(f"\nPrecision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Training accuracy: 1.0000
Validation accuracy: 1.0000
Test accuracy: 1.0000

Confusion Matrix:
[[ 40   0]
 [  0 273]]

Precision: 1.0000
Recall: 1.0000


In [79]:
joblib.dump(clf, 'sajda_hip.pkl')

['sajda_hip.pkl']

In [82]:
data = pd.read_csv('knee-train.csv')

X = data[['angle']]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression()


In [85]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

train_acc = accuracy_score(y_train, model.predict(X_train))
val_acc = accuracy_score(y_test, y_pred)

print('Train accuracy:', train_acc)
print('Validation accuracy:', val_acc)

confusion_matrix = confusion_matrix(y_test, y_pred)
print('Confusion matrix:')
print(confusion_matrix)

precision = precision_score(y_test, y_pred)
print('Precision:', precision)

recall = recall_score(y_test, y_pred)
print('Recall:', recall)

f1 = f1_score(y_test, y_pred)
print('F1-score:', f1)

Train accuracy: 1.0
Validation accuracy: 1.0
Confusion matrix:
[[ 75   0]
 [  0 545]]
Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [86]:
joblib.dump(model, 'sajda_knee.pkl')

['sajda_knee.pkl']